# Generating dataframe from the wiki data table

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
# print(soup.prettify())

In [4]:
table = soup.find('table')
# print(table.prettify())

Testing out how to fatch data from html using only one "tr"

In [5]:
l = table.tr
l

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

In [6]:
print(l.contents[1].text)
print(l.contents[3].text)
print(l.contents[5].text)

Postcode
Borough
Neighbourhood



By exploring above, now taking all the data from the table

In [7]:
postCode, borough, neighbourhood = [], [], []

for data in table.find_all('tr'):
    if data.contents[3].text != 'Not assigned':
        postCode.append(data.contents[1].text)
        borough.append(data.contents[3].text)
        neighbourhood.append(data.contents[5].text)

removing '\n' from the end of neighbourhood column

In [8]:
for i, neighbour in enumerate(neighbourhood):
    neighbourhood[i] = neighbour[:-1]

# print(neighbourhood)

Storing all the lists to the Pandas Dataframe

In [9]:
import pandas as pd

data = [postCode[1:], borough[1:], neighbourhood[1:]]
df = pd.DataFrame(data)
df = df.transpose()
df.head()

,0,1,2
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Changing the name of columns

In [10]:
df.rename(columns={0:'Postcode',
                  1:'Borough',
                  2:'Neighbourhood'},
         inplace=True)
print(df.columns)
unique_code = df['Postcode'].unique()
df.set_index('Postcode', inplace=True)
df.head()

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor


Checking how th loc workds 

In [11]:
df.loc['M6A']

,Borough,Neighbourhood
Postcode,,
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor


In [12]:
df.columns

Index(['Borough', 'Neighbourhood'], dtype='object')

Combine the data of same code and reset the index

In [13]:
for code in unique_code:
    data = df.loc[code]
    if data.size > 2:
        df.loc[code, 'Neighbourhood'] = ', '.join(data['Neighbourhood'])
        
# df.drop_duplicates('Postcode', inplace=True)
df.reset_index(inplace=True)
df.drop_duplicates('Postcode', inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


Get the sape of the final data frame

In [14]:
print(f'The shape of the final data frame is {df.shape}')

The shape of the final data frame is (103, 3)


In [ ]:
Finally Generated the Data Frame 

In [15]:
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


Tried working with teh "Geocoder", but had some trouble 

So, decided to grab Longitude and Latitude from the CSV file

In [16]:
lon_lat = pd.read_csv('Geospatial_Coordinates.csv')
lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
lon_lat.shape

(103, 3)

In [18]:
codes = lon_lat['Postal Code']


In [19]:
lon_lat.set_index("Postal Code", inplace=True)

In [20]:
df.set_index("Postcode", inplace=True)

In [21]:
df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park


In [22]:
for pc in codes:
    ll = lon_lat.loc[pc]
    df.loc[pc, 'Latitude'] = ll['Latitude']
    df.loc[pc, 'Longitude'] = ll['Longitude']

In [23]:
df.shape

(103, 4)

In [24]:
df.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [27]:
df.reset_index(inplace=True)

In [28]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
